# NN with LabelEncode Target Values
y shape is 1 column with 0-3

less successful than other

In [1]:
#Import Dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
#SOURCE: https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="Body_Performance", user='postgres', password='Cheese6132', host='localhost', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)



Connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [3]:
body_df = pd.read_sql('select * from body_performance', con=conn)
body_df.head()

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [4]:
#Closing the connection
conn.close()
print("Closed connection established to: ",data)


Closed connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [5]:
# OneHotEncoder or Get_dummies on Gender
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

encode1 = pd.DataFrame(enc.fit_transform(body_df.gender.values.reshape(-1,1)))

encode1.columns = enc.get_feature_names_out(['gender'])
encode1.head()


,gender_F,gender_M
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [6]:
#If keeping class/level in one column as Y, Encode with dictionary values
ranking_num = {"A":0, "B":1, "C":2, "D":3}
body_df["ranking_num"] = body_df["ranking"].apply(lambda x: ranking_num[x])
body_df.head()


,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking,ranking_num
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,2
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,0
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,2
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,1
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,1


In [7]:
#Using oneHotEncoder for class into 4 separate

#encode2 = pd.DataFrame(enc.fit_transform(body_df.ranking.values.reshape(-1,1)))

#encode2.columns = enc.get_feature_names_out(["ranking"])
#encode2.head()

In [8]:
#Drop columns and merge encoded df
#Ignoring encode2 for now
body_df = body_df.drop(["gender", "ranking"], axis=1)

body_df2 = body_df.merge(encode1, left_index=True, right_index=True)
body_df2.head()


,age,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking_num,gender_F,gender_M
0,27,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,2,0.0,1.0
1,25,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0,0.0,1.0
2,31,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,2,0.0,1.0
3,32,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1,0.0,1.0
4,28,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1,0.0,1.0


In [9]:
#Set X, y 
y = body_df2["ranking_num"].values
X = body_df2.drop("ranking_num", axis=1).values
    #X = X.drop("level_num", axis=1)

print(X.shape, y.shape)



(13393, 12) (13393,)


In [10]:
y[:10]

array([2, 0, 2, 1, 1, 1, 3, 1, 2, 1], dtype=int64)

In [11]:
#Split train/test data and scale using Standard Scaler at this time (mean of 0).  #Should we use MinMax(0-1)
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-0.7892366 , -0.91838934, -1.3348929 , ..., -0.56381633,
         1.3150467 , -1.3150467 ],
       [ 1.71061167, -0.50110528, -0.31377541, ..., -1.21956778,
        -0.76042927,  0.76042927],
       [-0.86276155,  0.16654921,  0.95843653, ...,  1.37821682,
        -0.76042927,  0.76042927],
       ...,
       [-1.00981144, -0.23881245,  1.05887432, ...,  0.49547448,
        -0.76042927,  0.76042927],
       [-1.15686134,  0.07117   , -0.21333763, ..., -0.41248907,
         1.3150467 , -1.3150467 ],
       [ 1.78413662, -1.20452697, -0.58160951, ..., -0.4881527 ,
        -0.76042927,  0.76042927]])

In [12]:
# Define the model - deep neural net...length of outputs/inputs
num_input_features = len(X_train_scaled[0])
num_output_features = 1  #len(y_train[0])
print(num_input_features)
print(num_output_features)

12
1


In [23]:
# Define the model - deep neural net
# Try with 2 hidden layers first but will try reduced. Look at different activation functions?
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 12




nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                312       
                                                                 
 dense_4 (Dense)             (None, 12)                300       
                                                                 
 dense_5 (Dense)             (None, 1)                 13        
                                                                 
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model binary vs categorical. Categorical for multi-output  --https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 703us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 2/200
314/314 [==============================] - 0s 687us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 3/200
314/314 [==============================] - 0s 684us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 4/200
314/314 [==============================] - 0s 684us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 5/200
314/314 [==============================] - 0s 674us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 6/200
314/314 [==============================] - 0s 685us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 7/200
314/314 [==============================] - 0s 807us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 8/200
314/314 [==============================] - 0s 696us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 9/200
314/314 [==============================] - 0s 681us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 10/200
314/314 [===============

Epoch 77/200
314/314 [==============================] - 0s 683us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 78/200
314/314 [==============================] - 0s 678us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 79/200
314/314 [==============================] - 0s 688us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 80/200
314/314 [==============================] - 0s 678us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 81/200
314/314 [==============================] - 0s 677us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 82/200
314/314 [==============================] - 0s 694us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 83/200
314/314 [==============================] - 0s 689us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 84/200
314/314 [==============================] - 0s 686us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 85/200
314/314 [==============================] - 0s 677us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 86/200
314/314 [======

314/314 [==============================] - 0s 681us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 153/200
314/314 [==============================] - 0s 701us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 154/200
314/314 [==============================] - 0s 696us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 155/200
314/314 [==============================] - 0s 669us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 156/200
314/314 [==============================] - 0s 735us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 157/200
314/314 [==============================] - 0s 713us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 158/200
314/314 [==============================] - 0s 671us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 159/200
314/314 [==============================] - 0s 662us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 160/200
314/314 [==============================] - 0s 694us/step - loss: 0.0000e+00 - accuracy: 0.2491
Epoch 161/200
314/314 [==========

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - loss: 0.0000e+00 - accuracy: 0.2523 - 149ms/epoch - 1ms/step
Loss: 0.0, Accuracy: 0.25231412053108215


In [24]:
# Define the model - deep neural net
# Try with 1 hidden layer keeping activation functions and node#

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer  #Will need to round or should this be sigmoid?
nn2.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 24)                312       
                                                                 
 dense_7 (Dense)             (None, 12)                300       
                                                                 
 dense_8 (Dense)             (None, 12)                156       
                                                                 
 dense_9 (Dense)             (None, 4)                 52        
                                                                 
Total params: 820
Trainable params: 820
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
314/314 [==============================] - 1s 808us/step - loss: -2.4873 - accuracy: 0.2742
Epoch 2/100
314/314 [==============================] - 0s 796us/step - loss: -8.8688 - accuracy: 0.2753
Epoch 3/100
314/314 [==============================] - 0s 783us/step - loss: -9.3163 - accuracy: 0.2632
Epoch 4/100
314/314 [==============================] - 0s 805us/step - loss: -9.4105 - accuracy: 0.2575
Epoch 5/100
314/314 [==============================] - 0s 802us/step - loss: -9.6383 - accuracy: 0.2491
Epoch 6/100
314/314 [==============================] - 0s 782us/step - loss: -9.6718 - accuracy: 0.2391
Epoch 7/100
314/314 [==============================] - 0s 777us/step - loss: -9.6972 - accuracy: 0.2461
Epoch 8/100
314/314 [==============================] - 0s 795us/step - loss: -9.7492 - accuracy: 0.3318
Epoch 9/100
314/314 [==============================] - 0s 800us/step - loss: -9.8841 - accuracy: 0.3077
Epoch 10/100
314/314 [==============================] - 0s 809us

314/314 [==============================] - 0s 778us/step - loss: -10.3175 - accuracy: 0.2427
Epoch 79/100
314/314 [==============================] - 0s 812us/step - loss: -10.3047 - accuracy: 0.2471
Epoch 80/100
314/314 [==============================] - 0s 792us/step - loss: -10.1842 - accuracy: 0.2660
Epoch 81/100
314/314 [==============================] - 0s 794us/step - loss: -10.1693 - accuracy: 0.3480
Epoch 82/100
314/314 [==============================] - 0s 777us/step - loss: -10.2832 - accuracy: 0.3511
Epoch 83/100
314/314 [==============================] - 0s 791us/step - loss: -10.2470 - accuracy: 0.3639
Epoch 84/100
314/314 [==============================] - 0s 775us/step - loss: -10.2839 - accuracy: 0.3256
Epoch 85/100
314/314 [==============================] - 0s 796us/step - loss: -10.3208 - accuracy: 0.3205
Epoch 86/100
314/314 [==============================] - 0s 782us/step - loss: -10.3332 - accuracy: 0.3094
Epoch 87/100
314/314 [==============================] - 0s 

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: -1.0335e+01 - accuracy: 0.2867 - 157ms/epoch - 1ms/step
Loss: -10.334750175476074, Accuracy: 0.28665274381637573


In [ ]:
feature_importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
#Accuracy score and confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

In [ ]:
# printing confusion matrix but error:      Shape of passed values is (4, 4), indices imply (2, 2) Needs to be resolved
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"])
display(cm_df)

In [ ]:
#Classification report for random forest

print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Need to test feature importances. THis is one way but plot is not showing up how I'd like. Need to adjust.

features = sorted(zip(X.columns, rf_model.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(10,200)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()